# Feature engineering: Aggregating sparse variant data

In this version of feature engineering, we follow the steps of the paper.


## Starting Dataset:
- Variant calls from 466 genes (SNVs, indel) and fusions in 3 genes.
  - Highly sparse data with few shared exact variants across samples.


## Feature Aggregation Approach:

- Data transformed to gene-level features
   - SNVs, indels and fusions (SV) handled separately.
   - Allele frequency (AF) values ignored
   - Binary representation indicating presence or absence of alteration.
   - example features: EZH2_SNV, PIM1_indel, BCL2_SV
   - MYD88 L273P mutation was singled out as a key driver for ABC DLBCL


In [61]:
# python module loading
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime



## SNV data


From the [supplemantatry](https://ehsantabari.com/public/goya_paper/GOYA_ctDNA_supplementary_methods_Leuk_Lymph_31Oct23.pdf) methods:

> ### Variant Filtering
>
> Common variants in the Single Nucleotide Polymorphism Database (dbSNP) or variants with an
> AF > 0.1% in any of 1000 Genomes project (1KG) or Exome Aggregation Consortium (ExaC)
> populations were filtered out unless they were reported by the Catalogue of Somatic Mutations in
> Cancer (COSMIC) or the Cancer Genome Atlas Program (TCGA). Variants that were detected in
> low complexity regions of the genome, as well as regions known to contain immunoglobulin
> genes, were filtered out. Variants that were commonly detected using our targeted panel on
> healthy normal plasma samples were also discarded. For this blacklist filter, 22 healthy donor
> samples were sequenced with our workflow, and any variants occurring in more than two healthy
> donors with more than five supporting reads were excluded from consideration in patients with
> DLBCL. Finally, variants with low coverage (a depth of < 25% of median sample depth) were
> filtered out. After filtering, 58961 SNVs and 2954 indels were detected in plasma samples.

In [55]:
# Loading SNV data
snv = pd.read_excel(
    "https://ehsantabari.com/public/goya_paper/Supplementary_Table_2A.xlsx"
)


print(snv.columns)


# Methods:
# "In one case, a variant within the MYD88 gene (L265P/L273P) was singled out as it is a known common driver for ABC DLBCL"
snv["feature_name"] = snv["Gene"].astype(str)
snv.loc[(snv["Gene"] == "MYD88") & (snv["AA"] == "p.Leu273Pro"), "feature_name"] = (
    "MYD88_L273P"
)

# chr17: 77451809-77452328 is misannotated; Fix Gene name
snv.loc[
    (snv["CHR"] == "chr17") & (snv["POS"] >= 77451809) & (snv["POS"] <= 77452328),
    "feature_name",
] = "SEPT9"

snv["feature_name"] = snv["feature_name"] + "_SNV"

snv

Index(['Sample number', 'CHR', 'POS', 'REF', 'ALT', 'Gene', 'CDS', 'AA',
       'NONSILENT', 'TOTDEPTH', 'VARDEPTH'],
      dtype='object')


,Sample number,CHR,POS,REF,ALT,Gene,CDS,AA,NONSILENT,TOTDEPTH,VARDEPTH,feature_name
0,1,chr1,2558453,C,A,TNFRSF14,c.289C>A,p.Gln97Lys,True,7672,5,TNFRSF14_SNV
1,1,chr1,23559151,A,G,ID3,c.276T>C,p.Pro92Pro,False,7234,5,ID3_SNV
2,1,chr1,85270733,T,C,BCL10,c.231A>G,p.Lys77Lys,False,7009,7,BCL10_SNV
3,1,chr1,474411,T,A,NTNG1,c.376T>A,p.Tyr126Asn,True,7408,4,NTNG1_SNV
4,1,chr1,203306270,T,G,BTG2,c.142+522T>G,NaN,False,6442,17,BTG2_SNV
...,...,...,...,...,...,...,...,...,...,...,...,...
58956,310,chr9,5534930,C,T,PDCD1LG2,c.241C>T,p.Pro81Ser,True,1889,899,PDCD1LG2_SNV
58957,310,chr10,89332613,A,G,IFIT3,c.-10A>G,NaN,False,1492,753,IFIT3_SNV
58958,310,chr14,105645665,C,A,RP11-731F5.2,n.105645665G>T,NaN,False,1670,365,RP11-731F5.2_SNV
58959,310,chr14,105774618,C,T,IGHG1,c.437-31548G>A,NaN,False,2666,93,IGHG1_SNV


The number of variants are post filtering

In [56]:
# For each sample, a list of genes with any non-silent somatic SNVs, indels or translocations was compiled,
# regardless of the loci of the variants within the gene; for example, SNV in EZH2, indel in PIM1 or translocation in BCL2.

snv_data = snv.groupby(["Sample number", "feature_name"]).size().reset_index()
snv_data["value"] = 1
snv_data = snv_data.pivot(
    index="Sample number", columns="feature_name", values="value"
).fillna(0)
snv_data = snv_data.reset_index()
snv_data

feature_name,Sample number,ABCB11_SNV,AC004623.2_SNV,AC007250.4_SNV,AC007386.4_SNV,AC0142.1_SNV,AC092170.1-AC009312.1_SNV,AC096579.13_SNV,AC118562.1-RP11-427M20.1_SNV,AC245028.1_SNV,...,ZEB2_SNV,ZFP36L1_SNV,ZFP42_SNV,ZMYM6_SNV,ZNF160_SNV,ZNF577_SNV,ZNF578_SNV,ZNF608_SNV,ZNF649_SNV,ZNF678_SNV
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,306,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,307,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,308,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
308,309,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Indel data

We are expecting to find that 
> ... _2954 indels were detected in plasma samples.

In [57]:
# Loading SNV data
indel = pd.read_excel(
    "https://ehsantabari.com/public/goya_paper/Supplementary_Table_2B.xlsx"
)

print(indel.columns)

# There is a space after the sample number column name: Fix it
indel.rename(columns={"Sample number ": "Sample number"}, inplace=True)

indel["feature_name"] = indel["Gene"] + "_INDEL"

indel

Index(['Sample number ', 'CHR', 'POS', 'REF', 'ALT', 'Gene', 'CDS', 'AA',
       'TOTDEPTH', 'VARDEPTH'],
      dtype='object')


,Sample number,CHR,POS,REF,ALT,Gene,CDS,AA,TOTDEPTH,VARDEPTH,feature_name
0,1,chr2,96144233,GA,G,DUSP2,c.650delT,p.Phe217fs,9341,19,DUSP2_INDEL
1,1,chr8,127738385,GC,G,MYC,c.173delC,p.Pro58fs,7114,15,MYC_INDEL
2,1,chr12,49022330,TC,T,KMT2D,c.404delG,p.Arg135fs,6071,13,KMT2D_INDEL
3,1,chr16,3729196,GCT,G,CREBBP,c.5849_5850delAG,p.Gln1950fs,3073,12,CREBBP_INDEL
4,1,chr17,65056349,TCG,T,G13,c.243_244delCG,p.Glu82fs,2864,7,G13_INDEL
...,...,...,...,...,...,...,...,...,...,...,...
2949,309,chrX,74742737,GA,G,NEXMIF,c.1819delT,p.Ser607fs,4214,9,NEXMIF_INDEL
2950,309,chrX,87518322,GT,G,KLHL4,c.422+8delT,NaN,2434,12,KLHL4_INDEL
2951,310,chr6,137871203,TCC,T,TNFAIP3,c.-15-9_-15-8delCC,NaN,739,6,TNFAIP3_INDEL
2952,310,chr17,7673747,CT,C,TP53,c.872delA,p.Lys291fs,5104,12,TP53_INDEL


In [58]:
# For each sample, a list of genes with any non-silent somatic SNVs, indels or translocations was compiled,
# regardless of the loci of the variants within the gene; for example, SNV in EZH2, indel in PIM1 or translocation in BCL2.

indel_data = indel.groupby(["Sample number", "feature_name"]).size().reset_index()
indel_data["value"] = 1
indel_data = indel_data.pivot(
    index="Sample number", columns="feature_name", values="value"
).fillna(0)
indel_data = indel_data.reset_index()
indel_data

feature_name,Sample number,ABCB11_INDEL,AC072022.1_INDEL,ACAD8_INDEL,ACTA2_INDEL,ACTB_INDEL,ACTG1_INDEL,ACVR2A_INDEL,ADAMTS1_INDEL,AL928742.1_INDEL,...,TNFAIP3_INDEL,TNFRSF14_INDEL,TP53_INDEL,TSPOAP1-AS1_INDEL,XBP1_INDEL,ZEB2_INDEL,ZFP36L1_INDEL,ZFP42_INDEL,ZNF608_INDEL,ZNF649_INDEL
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287,306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Fusion data

We are expecting to find that 

> In addition, 170 translocations of BCL2, BCL6 and MYC genes were detected (Table S2).

In [ ]:
# Loading SNV data
fusion = pd.read_excel(
    "https://ehsantabari.com/public/goya_paper/Supplementary_Table_2C.xlsx"
)
print(fusion.columns)

fusion["feature_name"] = fusion["Gene"] + "_SV"

fusion

Index(['Sample number', 'Gene', 'Gene (other)', 'Est_Type', 'Total depth',
       'Break depth', 'Break support1', 'Break support2',
       'Proper pair support'],
      dtype='object')


,Sample number,Gene,Gene (other),Est_Type,Total depth,Break depth,Break support1,Break support2,Proper pair support,feature_name
0,1,BCL6,IGHG2,TRA,46462,33,21,30,31,BCL6_SV
1,2,BCL6,IRF8,TRA,23682,3315,2731,1,3308,BCL6_SV
2,5,BCL6,IRF1,TRA,19445,90,54,41,90,BCL6_SV
3,6,BCL2,IGHJ6,TRA,27840,288,2,636,270,BCL2_SV
4,7,BCL2,IGLJ2,TRA,5201,657,26,4,648,BCL2_SV
...,...,...,...,...,...,...,...,...,...,...
165,303,BCL6,MIR4537,TRA,18066,1213,988,2,1190,BCL6_SV
166,303,MYC,IGHG4,TRA,21952,743,467,3,737,MYC_SV
167,306,BCL2,MIR4537,TRA,15650,2432,2015,1542,2410,BCL2_SV
168,308,BCL2,PHLPP1,DEL,23209,1690,1,1413,1688,BCL2_SV


In [60]:
# For each sample, a list of genes with any non-silent somatic SNVs, indels or translocations was compiled,
# regardless of the loci of the variants within the gene; for example, SNV in EZH2, indel in PIM1 or translocation in BCL2.

fusion_data = fusion.groupby(["Sample number", "feature_name"]).size().reset_index()
fusion_data["value"] = 1
fusion_data = fusion_data.pivot(
    index="Sample number", columns="feature_name", values="value"
).fillna(0)
fusion_data = fusion_data.reset_index()
fusion_data

feature_name,Sample number,BCL2_SV,BCL6_SV,MYC_SV
0,1,0.0,1.0,0.0
1,2,0.0,1.0,0.0
2,5,0.0,1.0,0.0
3,6,1.0,0.0,0.0
4,7,1.0,0.0,0.0
...,...,...,...,...
114,298,0.0,1.0,0.0
115,303,0.0,1.0,1.0
116,306,1.0,0.0,0.0
117,308,1.0,0.0,0.0


## Merge the dataset and create a data file

The merged data will be stored locally for later use as `goya_data-gene_noAf-(datetime).csv`

In [65]:
# merge the three snv, indel and fusion data and save it to a file goya_data-gene_noAf-(datetime).csv 
goya_data = pd.merge(snv_data, indel_data, on="Sample number", how="outer")
goya_data = pd.merge(goya_data, fusion_data, on="Sample number", how="outer")
goya_data = goya_data.fillna(0)
goya_data.shape

(310, 675)

In [66]:
# add date time to the file name to avoid overwriting
# use a file name frirendly format, no need to include the seconds in the file name
# now = datetime.datetime.now()
now = datetime.datetime.now().strftime("%y_%m_%d_%H_%M")  
goya_data.to_csv(f"goya_data-gene_noAf-{now}.csv", index=False)